##BIBLIOTECA MATRICIAL
Hecho por: **Javier Verde Marín**

En este notebook se van a implementar operaciones básicas con dos matrices utilizando mapreduce
* Suma
* Resta
* Multiplicación
* División

Primero vamos a importar las bibliotecas necesarias y definir las matrices

In [0]:
import functools
import numpy as np
#Comentar uno u otro para poder realizar la división (en el primer y segundo caso de B, el determinante daría 0)
#Casos que he probado de A
#A = np.array([[1,2,3]])
A = np.array([[1,2,3],[1,2,3],[1,2,3]])
#Casos que he probado de B
B = np.array([[3,2,1],[3,2,1],[3,2,1]])
#B = np.array([[3,2],[3,2],[3,2]])
#B = np.array([[5,2,1],[3,11,1],[3,2,7]])

####SUMA

En este caso he usado como prueba:
 * A = np.array([[1,2,3],[1,2,3],[1,2,3]])
 * B = np.array([[3,2,1],[3,2,1],[3,2,1]])

In [0]:
def suma(a, b):
   if len(a)==len(b) and len(a[0])==len(b[0]):
     rdda = sc.parallelize(a, len(a[0]))
     rddb = sc.parallelize(b, len(b[0]))
     rdd = rdda.zip(rddb)
     rddresult = rdd.map(lambda x: (x[0]+x[1]))
     return np.array(rddresult.collect()).reshape(len(a),len(a[0]))
   else: print("No se puede")
      
suma(A,B)

Out[29]: array([[4, 4, 4],
 [4, 4, 4],
 [4, 4, 4]])

####RESTA

En este caso he usado como prueba:
 * A = np.array([[1,2,3],[1,2,3],[1,2,3]])
 * B = np.array([[3,2,1],[3,2,1],[3,2,1]])

In [0]:
def resta (a, b):
  if len(a)==len(b) and len(a[0])==len(b[0]):
     rdda = sc.parallelize(a, len(a[0]))
     rddb = sc.parallelize(b, len(b[0]))
     rdd = rdda.zip(rddb)
     rddresult = rdd.map(lambda x: (x[0]-x[1]))
     return np.array(rddresult.collect()).reshape(len(a),len(a[0]))
  else: print("No se puede")

resta(A,B)

Out[30]: array([[-2, 0, 2],
 [-2, 0, 2],
 [-2, 0, 2]])

####MULTIPLICACIÓN

En este caso he usado como prueba:
 * A = np.array([[1,2,3],[1,2,3],[1,2,3]])
 * B = np.array([[3,2],[3,2],[3,2]])

In [0]:
def multiplicacion (a, b):
  if len(a[0]) == len(b):
   result = np.empty((len(a[0]), len(b)))
   b_aux = np.array(b).transpose().reshape(len(b[0]), -1) 
   rddb = sc.parallelize(b_aux, len(b_aux))
   for i in range(len(a)):
     a_aux =  np.array([a[i],]*len(b[0])).reshape(len(b[0]), -1)
     rdda = sc.parallelize(a_aux, len(a_aux))
     rddc = rdda.zip(rddb)
     rddc2 = rddc.map(lambda x: sum((x[0] * x[1])))
     if i == 0: result = np.array(rddc2.collect())
     else: result = np.concatenate((result,np.array(rddc2.collect())))
   return result.reshape(len(a),len(b[0]))
  else: print("No se cumplen las condiciones de multiplicación")

multiplicacion(A,B)

Out[24]: array([[18, 12],
 [18, 12],
 [18, 12]])

####DIVISIÓN

Como la división de dos matrices es la matriz por la inversa de la segunda matriz, vamos a aprovecharnos de la función anterior
En la division, la matrix B debe ser cuadrada

En este caso he usado como prueba:
 * A = np.array([[1,2,3],[1,2,3],[1,2,3]])
 * B = np.array([[5,2,1],[3,11,1],[3,2,7]])

In [0]:
def division (a, b):
  if len(b) == len(b[0]) and np.linalg.det(b) != 0:
   invert = np.linalg.inv(b)
   if len(a[0]) == len(invert):
     result = np.empty((len(a[0]), len(invert)))
     b_aux = np.array(invert).transpose().reshape(len(invert[0]), -1) 
     rddb = sc.parallelize(b_aux, len(b_aux))
     for i in range(len(a)):
       a_aux =  np.array([a[i],]*len(b[0])).reshape(len(a), -1)
       rdda = sc.parallelize(a_aux, len(a[0]))
       rddc = rdda.zip(rddb)
       rddc2 = rddc.map(lambda x: sum((x[0] * x[1])))
       if i == 0: result = np.array(rddc2.collect())
       else: result = np.concatenate((result,np.array(rddc2.collect())))
     return result.reshape(len(a),len(invert[0]))
   else: print("No se puede, no se cumplen con los requisitos de multiplicación")
  else: print("No se puede, matrix B no es cuadrada o su determinante es 0")
    
division(A,B)

Out[27]: array([[-0.13461538, 0.12820513, 0.42948718],
 [-0.13461538, 0.12820513, 0.42948718],
 [-0.13461538, 0.12820513, 0.42948718]])